In [1]:
!nvidia-smi

Wed Apr 20 06:29:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive

Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [3]:
!ls
%cd drive/MyDrive/VIPLAB/adl_bert/
!ls

drive  sample_data
/content/drive/MyDrive/VIPLAB/adl_bert
ADL_BERT.ipynb	data


In [4]:
!pip install transformers
!pip install tqdm

     |████████████████████████████████| 4.0 MB 8.5 MB/s 
     |████████████████████████████████| 596 kB 44.8 MB/s 
     |████████████████████████████████| 895 kB 49.6 MB/s 
     |████████████████████████████████| 6.6 MB 37.2 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Import

In [28]:
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_cosine_schedule_with_warmup
from tqdm.auto import tqdm

data_path = './data/intent/'

def read_data(data_path, mode, intent2idx, idx2intent):
    data_text = []
    data_intent = []
    data_id = []
    with open(f'{data_path}{mode}.json') as f:
        datas = json.load(f)
        for data in datas:
            data_text.append(data['text'])
            if mode != 'test':
                data_intent.append(data['intent'])
            data_id.append(data['id'])
    if mode == 'train':
        idx = 0
        for intent in data_intent:  
            if intent in intent2idx:
                continue
            else:
                intent2idx[intent] = idx
                idx2intent[idx] = intent
                idx += 1
    for idx, intent in enumerate(data_intent):
        data_intent[idx] = intent2idx[intent]

    return data_text, data_intent, data_id, intent2idx, idx2intent
intent2idx = {}
idx2intent = {}
train_text, train_intent, train_id, intent2idx, idx2intent = read_data(data_path, 'train', intent2idx, idx2intent)
valid_text, valid_intent, valid_id, _, _ = read_data(data_path, 'eval', intent2idx, idx2intent)
test_text, _, test_id, _, _ = read_data(data_path, 'test', intent2idx, idx2intent)

print(len(train_text))
print(len(valid_text))
print(len(test_text))
print(train_intent[:10])
print(valid_intent[:10])

15000
3000
4500
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[25, 110, 97, 34, 113, 108, 75, 135, 16, 19]


## Dataset

In [29]:
class ADLDataset(Dataset):
    def __init__(self, input_dict, mode, names, y= None):
        self.mode = mode
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.names = names
        self.y = y
    def __getitem__(self, idx):
        input_id = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        if self.mode != 'test':
            label = self.y[idx]
            return input_id, tokentype, attentionmask, label
        else:
            return input_id, tokentype, attentionmask, self.names[idx]
    def __len__(self):
        return len(self.input_ids)

  
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_input_dict = tokenizer.batch_encode_plus(train_text,
                                              add_special_tokens = True,
                                              max_length = 512,
                                              truncation = True,                ##是否截斷
                                              return_special_tokens_mask = True,
                                              pad_to_max_length = True,
                                              return_tensors = 'pt')
BATCH_SIZE = 4
train_y = np.array(train_intent)
trainset = ADLDataset(train_input_dict, 'train', train_id, train_y)  
trainloader = DataLoader(trainset, batch_size = BATCH_SIZE, shuffle = True)  


valid_input_dict = tokenizer.batch_encode_plus(valid_text,
                                              add_special_tokens = True,
                                              max_length = 512,
                                              truncation = True,                ##是否截斷
                                              return_special_tokens_mask = True,
                                              pad_to_max_length = True,
                                              return_tensors = 'pt')
valid_y = np.array(valid_intent)       
validset = ADLDataset(valid_input_dict, 'valid', valid_id, valid_y)  
validloader = DataLoader(validset, batch_size = BATCH_SIZE, shuffle = False)  



test_input_dict = tokenizer.batch_encode_plus(test_text,
                                              add_special_tokens = True,
                                              max_length = 512,
                                              truncation = True,                ##是否截斷
                                              return_special_tokens_mask = True,
                                              pad_to_max_length = True,
                                              return_tensors = 'pt')    
testset = ADLDataset(test_input_dict, 'test', test_id)  
testloader = DataLoader(testset, batch_size = BATCH_SIZE, shuffle = False)  

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

NUM_LABELS = len(intent2idx)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = NUM_LABELS)
model = model.to(device)

model_path = './BERT_intent.ckpt'

EPOCHS = 5
best_acc = 0.0
stop_count = 0
lr = 1e-4

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
accum_iter = 16
total_step = len(trainset) * EPOCHS // (BATCH_SIZE * accum_iter)
warmup_step = total_step * 0.12
best_acc = 0.0
scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_step, total_step)
model = model.to(device)
model.train()

print('Start Training')
for epoch in range(EPOCHS):
    train_loss = []
    correct = 0
    optimizer.zero_grad()
    with torch.set_grad_enabled(True):
        for (batch_idx,data) in enumerate(tqdm(trainloader)):
            tokens_tensors ,  segments_tensors , masks_tensors , labels  = data
            tokens_tensors = tokens_tensors.to(device)
            segments_tensors = segments_tensors.to(device)
            masks_tensors = masks_tensors.to(device)
            labels = labels.to(device)
    #         print(data)
            bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors,
                                labels = labels)
            logits = bert_outputs.logits

            loss = criterion(logits,labels) 

            train_loss.append(loss.detach().item())
            loss = loss / accum_iter
            loss.backward()

            acc = (logits.cpu().argmax(dim=-1) == labels.cpu()).float().mean()
            correct += sum(logits.cpu().argmax(dim=-1) == labels.cpu()).item()

        ## Update weights
            if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(trainloader)):
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

    train_loss = sum(train_loss) / len(train_loss)
    train_acc = correct / len(trainset)

    ## Print the infor
    print(f"[ Train | {epoch + 1:03d}/{EPOCHS:03d} ] Loss = {train_loss:.5f}, Acc = {train_acc:.5f}")

    ## ---------- Validation ----------
    model.eval()
    valid_loss = []
    correct = 0

    with torch.no_grad():
        for (batch_idx,data) in enumerate(tqdm(validloader)):
            tokens_tensors ,  segments_tensors , masks_tensors , labels  = data
            tokens_tensors = tokens_tensors.to(device)
            segments_tensors = segments_tensors.to(device)
            masks_tensors = masks_tensors.to(device)
            labels = labels.to(device)
    #         print(data)
            bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors,
                                labels = labels)
            logits = bert_outputs.logits
            labels = labels.to(device)

            loss = criterion(logits,labels) 

            valid_loss.append(loss.detach().item())
            loss = loss / accum_iter
            # loss.backward()

            acc = (logits.cpu().argmax(dim=-1) == labels.cpu()).float().mean()
            correct += sum(logits.cpu().argmax(dim=-1) == labels.cpu()).item()

    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = correct / len(validset)

    ## Print the info
    print(f"[ Valid | {epoch + 1:03d}/{EPOCHS:03d} ] Loss = {valid_loss:.5f}, Acc = {valid_acc:.5f}")

    ## Save Best Model
    if valid_acc >= best_acc:
        torch.save(model.state_dict(),model_path)
        print(f'Save model with acc: {valid_acc:.5f}')
        best_acc = valid_acc

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Start Training


  0%|          | 0/3750 [00:00<?, ?it/s]

[ Train | 001/005 ] Loss = 3.38105, Acc = 0.43220


  0%|          | 0/750 [00:00<?, ?it/s]

...

[ Train | 004/005 ] Loss = 0.04513, Acc = 0.99780


  0%|          | 0/750 [00:00<?, ?it/s]

[ Valid | 004/005 ] Loss = 0.16743, Acc = 0.96867
Save model with acc: 0.96867


  0%|          | 0/3750 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## Inference

In [37]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = NUM_LABELS)
model.load_state_dict(torch.load(model_path))
model = model.to(device)
model.eval()

preds = torch.tensor([])
pred_names = []

with torch.no_grad():
    for (batch_idx,data) in enumerate(tqdm(testloader)):
        tokens_tensors ,  segments_tensors , masks_tensors , names  = data
        tokens_tensors = tokens_tensors.to(device)
        segments_tensors = segments_tensors.to(device)
        masks_tensors = masks_tensors.to(device)
        
        bert_outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
        logits = bert_outputs.logits
        pred = torch.argmax(logits.cpu(), dim=-1)
        if len(preds) == 0:
            preds = pred
            pred_names += list(names)
        else:
            preds = torch.cat((preds, pred), 0)
            pred_names += list(names)

with open('pred.csv', 'w') as f:
    f.write('id,intent\n')
    for i, y in enumerate(preds):
        f.write('{},{}\n'.format(pred_names[i],idx2intent[y.item()]))



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/1125 [00:00<?, ?it/s]